In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns
import pickle
import time
import gc
from tqdm import tqdm, tqdm_notebook

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [2]:
import os
import gensim

In [3]:
DF_NUM = 12311

In [4]:
models = []
for idx, file in enumerate(os.listdir('./feature/')):
    models.append(
        gensim.models.KeyedVectors.load_word2vec_format(
            './w2v/%s.model' % (file[:-4]), binary=False))
    print('finish', file)

finish dict(full).txt
finish dict(noNumLetter).txt
finish dict(noStop).txt
finish dict(noStopnoNumLetter).txt


In [5]:
words = []
for file in os.listdir('./feature/'):
    tmp = []
    with open('./feature/' + file, 'r', encoding='utf-8') as f:
        for line in f:
            tmp.append(line[:-1].split(' '))
    words.append(tmp)
    print('finish', file)

finish dict(full).txt
finish dict(noNumLetter).txt
finish dict(noStop).txt
finish dict(noStopnoNumLetter).txt


In [6]:
w2v = []
for idx, sents in tqdm_notebook(enumerate(words)):
    tmps = []
    for sent in sents:
        tmp = [0] * 64
        words_num = 0
        for word in sent:
            if word in models[idx]:
                tmp += models[idx][word]
                words_num += 1
        tmp = list(np.array(tmp) / words_num)
        tmps.append(tmp)
    w2v.append(tmps)

A Jupyter Widget

## XGB

In [7]:
import xgboost as xgb

In [8]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test_public.csv')

train.shape
train.head()
test.shape
test.head()

(9947, 5)

,content_id,content,subject,sentiment_value,sentiment_word
0,vUXizsqexyZVRdFH,因为森林人即将换代，这套系统没必要装在一款即将换代的车型上，因为肯定会影响价格。,价格,0,影响
1,4QroPd9hNfnCHVt7,四驱价格貌似挺高的，高的可以看齐XC60了，看实车前脸有点违和感。不过大众的车应该不会差。,价格,-1,高
2,QmqJ2AvM5GplaRyz,斯柯达要说质量，似乎比大众要好一点，价格也低一些，用料完全一样。我听说过野帝，但没听说过你说...,价格,1,低
3,KMT1gFJiU4NWrVDn,这玩意都是给有钱任性又不懂车的土豪用的，这价格换一次我妹夫EP020可以换三锅了,价格,-1,有钱任性
4,nVIlGd5yMmc37t1o,17价格忒高，估计也就是14-15左右。,价格,-1,高


(2364, 2)

,content_id,content
0,XuPwKCnA2fqNh5vm,欧蓝德，价格便宜，森林人太贵啦！
1,2jNbDn85goX3IuPE,楼主什么时候提的车，南昌优惠多少啊
2,hLgEADQ8sUnvGFK9,吉林，2.5优惠20000，送三年九次保养，贴膜
3,nZmM7LQsfr03wUaz,便宜2万的豪华特装，实用配制提升，优惠还给力，确实划算。
4,pwd8MnrthDqLZafe,如果实在想买就等车展期间，优惠2万，我24.98万入的2.5豪


In [9]:
sub_dict = dict(
    zip(list(train.subject.unique()), range(len(train.subject.unique()))))
sentiment_dict = dict(
    zip(
        list(train.sentiment_value.unique()),
        range(len(train.sentiment_value.unique()))))

train.subject = train.subject.map(sub_dict)
train.sentiment_value = train.sentiment_value.map(sentiment_dict)

In [10]:
from sklearn.cross_validation import train_test_split

target = ['subject', 'sentiment_value']
train_X, val_X, train_y, val_y = train_test_split(
    train, train[target], test_size=0.2, random_state=2018)

train_X.shape
val_X.shape

C:\Users\ZERO\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(7957, 5)

(1990, 5)

In [15]:
idx = 3
y_idx = 1
y_num = [10, 3]

In [16]:
train_matrix = np.array(w2v[idx])[list(train_X.index)]
val_matrix = np.array(w2v[idx])[list(val_X.index)]
test_matrix = np.array(w2v[idx][-test.shape[0]:])

In [17]:
params = {
    'objective': 'multi:softmax',
    'eta': 0.1,
#     'colsample_bytree': 0.3,
#     'subsample':0.9,
#     'min_child_weight': 17,
    'max_depth': 5,
    'num_class': y_num[y_idx],
    'gamma': 0.1,
    'seed': 2018,
    'missing': -1,
    'n_thread': 8
}

dtrain = xgb.DMatrix(train_matrix, label=train_y.iloc[:, y_idx])
dval = xgb.DMatrix(val_matrix, label=val_y.iloc[:, y_idx])
model = xgb.train(
    params,
    dtrain,
    num_boost_round=20000,
    early_stopping_rounds=50,
    evals=[(dtrain, 'train'), (dval, 'val')],
    verbose_eval=100)

num_round = int(model.best_iteration * 1.1)
dtrain = xgb.DMatrix(np.concatenate([train_matrix, val_matrix], axis=0), label=pd.concat([train_y, val_y], axis=0).iloc[:, y_idx])
dtest = xgb.DMatrix(test_matrix)
bst = xgb.train(params, dtrain, num_round)

pred = bst.predict(dtest)

[0]	train-merror:0.294332	val-merror:0.328141
Multiple eval metrics have been passed: 'val-merror' will be used for early stopping.

Will train until val-merror hasn't improved in 50 rounds.
[100]	train-merror:0.181601	val-merror:0.296985
Stopping. Best iteration:
[129]	train-merror:0.155963	val-merror:0.289447



In [18]:
if y_idx == 0:
    test['subject'] = pred
    test['subject'] = test['subject'].astype(int)
if y_idx == 1:
    test['sentiment_value'] = pred
    test['sentiment_value'] = test['sentiment_value'].astype(int)

In [19]:
sub_dict_rev = dict(zip(sub_dict.values(), sub_dict.keys()))
sentiment_dict_rev = dict(zip(sentiment_dict.values(), sentiment_dict.keys()))

test['subject'] = test['subject'].map(sub_dict_rev)
test['sentiment_value'] = test['sentiment_value'].map(sentiment_dict_rev)
test['sentiment_word'] = ''

In [20]:
test.head()

,content_id,content,subject,sentiment_value,sentiment_word
0,XuPwKCnA2fqNh5vm,欧蓝德，价格便宜，森林人太贵啦！,价格,0,
1,2jNbDn85goX3IuPE,楼主什么时候提的车，南昌优惠多少啊,价格,0,
2,hLgEADQ8sUnvGFK9,吉林，2.5优惠20000，送三年九次保养，贴膜,价格,0,
3,nZmM7LQsfr03wUaz,便宜2万的豪华特装，实用配制提升，优惠还给力，确实划算。,价格,0,
4,pwd8MnrthDqLZafe,如果实在想买就等车展期间，优惠2万，我24.98万入的2.5豪,价格,0,


In [21]:
test.groupby(
    ['subject'], as_index=False)['content_id'].agg({
        'subject_num': 'count'
    })
test.groupby(
    ['sentiment_value'], as_index=False)['content_id'].agg({
        'sentiment_value_num':
        'count'
    })

,subject,subject_num
0,价格,342
1,内饰,83
2,动力,860
3,外观,74
4,安全性,105
5,操控,222
6,油耗,230
7,空间,63
8,舒适性,208
9,配置,177


,sentiment_value,sentiment_value_num
0,-1,73
1,0,2210
2,1,81


In [22]:
test[['content_id', 'subject', 'sentiment_value', 'sentiment_word']].to_csv('./result/xgb_w2v64_idx3(min_count=0).csv', index=False, encoding='UTF-8')